# **Step2 제공파일_AI면접관 Agent_with Gradio**

## **1. 환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project_genai)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (2) 라이브러리

In [ ]:
!pip install langchain_openai langchain_core langchain-community -q
!pip install -U langgraph
!pip install PyMuPDF python-docx gradio -q

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [ ]:
!pip install chromadb
!apt-get update
!apt-get install -y fonts-nanum
!pip install reportlab

### (3) OpenAI API Key 확인
* api_key.txt 파일에 다음의 키를 등록하세요.
    * OPENAI_API_KEY

In [ ]:
import os

def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/langchain/'
# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

In [ ]:
print(os.environ['OPENAI_API_KEY'][:30])
path = '/content/drive/MyDrive/kt aivle/한경훈 대위햄/AI_미니프로젝트 2차_3일차_실습자료/'

sk-proj-shXQh5RT2uL_Khb0wD_xrm


## **2. App.py**

* 아래 코드에, Step1 혹은 고도화 된 Step2 파일 코드를 붙인다.
    * 라이브러리
    * 함수들과 그래프
* Gradio 코드는 그대로 사용하거나 일부 수정 가능

In [ ]:
%%writefile app.py

####### 여러분의 함수와 클래스를 모드 여기에 붙여 넣읍시다. #######
## 1. 라이브러리 로딩 ---------------------------------------------
import pandas as pd
import numpy as np
import os
import openai
import random
import ast
import fitz
from docx import Document
import json
import matplotlib.pyplot as plt
from math import pi
from langchain_core.messages import SystemMessage


from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.cidfonts import UnicodeCIDFont

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from typing import Annotated, Literal, Sequence, TypedDict, List, Dict
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.output_parsers import CommaSeparatedListOutputParser
from langgraph.graph import StateGraph, END

## ---------------- 1단계 : 사전준비 ----------------------

# 1) 파일 입력 --------------------
def extract_text_from_file(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        doc = fitz.open(file_path)
        text = "\n".join(page.get_text() for page in doc)
        doc.close()
        return text
    elif ext == ".docx":
        doc = Document(file_path)
        return "\n".join(p.text for p in doc.paragraphs if p.text.strip())
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 허용됩니다.")

# 2) State 선언 --------------------
class InterviewState(TypedDict):
    # 고정 정보
    resume_text: str
    resume_summary: str
    resume_keywords: List[str]
    question_strategy: Dict[str, Dict]

    # 인터뷰 로그
    current_question: str
    current_answer: str
    current_strategy: str
    conversation: List[Dict[str, str]]
    evaluation : List[Dict[str, str]]
    next_step : str
    reflection_status : str
    final_report : str
    pdf_path : str

# 3) resume 분석 --------------------
def analyze_resume(state: InterviewState) -> InterviewState:
    llm = ChatOpenAI(model_name='gpt-4.1-mini')

    # 이력서 핵심 요약
    summary_prompt = ChatPromptTemplate.from_messages([
        ("system", "너는 이력서 분석 전문가야."),
        ("human", "면접 질문을 위해 {resume_text}의 내용을 정확하게 파악하고 싶어. {resume_text}의 핵심 내용을 10문장 이내로 요약해줘.")
    ])

    summary_messages = summary_prompt.format_messages(resume_text = state['resume_text'])
    summary_response = llm.invoke(summary_messages)
    resume_summary = summary_response.content.strip()

    # 이력서 주요 키워드 추출
    keyword_prompt = ChatPromptTemplate.from_messages([
        ("system", "너는 이력서 분석 전문가야."),
        ("human", "면접 질문의 방향성을 잡을 수 있게 {resume_text}에서 주요 키워드 10개 이내로 뽑아서 쉼표로 구분해서 나열해줘.")
    ])
    keyword_messages = keyword_prompt.format_messages(resume_text=state['resume_text'])
    keyword_response = llm.invoke(keyword_messages)
    resume_keywords = [kw.strip() for kw in keyword_response.content.split(',')]

    return {
        **state,
        "resume_summary": resume_summary,
        "resume_keywords": resume_keywords,
    }

# 4) 질문 전략 수립 --------------------

def generate_question_strategy(state: InterviewState) -> InterviewState:
    llm = ChatOpenAI(model_name='gpt-4.1-mini')

    # 질문 전략 수립
    s_msg = '너는 10년 이상 경력의 면접관이야.'
    h_msg = '''
    {resume_summary}와 {resume_keywords}를 바탕으로 개인화된 면접 질문을 뽑을 거야.
    아래 3가지 분야에 대해 각각 "질문 방향"과 "예시 질문"을 포함한 전략을 JSON 형태로 작성해줘.

    반드시 아래 형식을 따르세요:
    {{
        "경력 및 경험": {{
            "질문 방향": "문장으로 작성",
            "예시 질문": ["질문1", "질문2"]
        }},
        "동기 및 커뮤니케이션": {{
            "질문 방향": "문장으로 작성",
            "예시 질문": ["질문1", "질문2"]
        }},
        "논리적 사고": {{
            "질문 방향": "문장으로 작성",
            "예시 질문": ["질문1", "질문2"]
        }}
    }}

    '''

    strategy_prompt = ChatPromptTemplate.from_messages([
        ("system", s_msg),
        ("human", h_msg)
    ])

    strategy_messages = strategy_prompt.format_messages(resume_summary = state['resume_summary'], resume_keywords = state['resume_keywords'])
    strategy_response = llm.invoke(strategy_messages)
    strategy_output = strategy_response.content.strip()

    # JSON 파싱
    try:
        strategy_dict = json.loads(strategy_output)
    except json.JSONDecodeError:
        print("⚠️ JSON 파싱 실패 - 문자열 그대로 저장합니다.")
        strategy_dict = {"raw_output": strategy_output}

    return {
        **state,
        "question_strategy": strategy_dict
    }

# 5) 1단계 하나로 묶기 --------------------

def preProcessing_Interview(file_path: str) -> InterviewState:
    # 파일 입력
    resume_text = extract_text_from_file(file_path)

    # state 초기화
    initial_state: InterviewState = {
        "resume_text": resume_text,
        "resume_summary": '',
        "resume_keywords": [],
        "question_strategy": {},

        "current_question": '',
        "current_answer": '',
        "current_strategy": '',
        "conversation": [],
        "evaluation": [],
        "next_step" : '',
        "reflection_status": '',
        "final_report" : '',
        "pdf_path" : ''
    }

    # Resume 분석
    state = analyze_resume(initial_state)

    # 질문 전략 수립
    state = generate_question_strategy(state)

    # 첫번째 질문 생성
    selected_question = state['question_strategy']['경력 및 경험']['예시 질문'][0]

    return {
            **state,
            "current_question": selected_question,
            "current_strategy": "경력 및 경험"
            }


## ---------------- 2단계 : 면접 Agent ----------------------

# 1) 답변 입력 --------------------
def update_current_answer(state: InterviewState, user_input) -> InterviewState:
    return {
        **state,
        "current_answer": user_input.strip()
    }

# 2) 답변 평가 --------------------
def evaluate_answer(state: InterviewState) -> InterviewState:
    # 답변 평가
    llm = ChatOpenAI(model_name='gpt-4.1-mini')

    s_msg = '너는 10년 이상 경력의 면접관이야.'
    h_msg = '''
    아래 질문과 답변을 보고 면접 답변이 적절한지 아래 2가지 항목을 각각 상, 중, 하로 평가해줘.

    질문: {current_question}
    답변: {current_answer}

    평가 항목:
    1. 질문과의 관련성
    - 상: 질문의 핵심 의도에 정확히 부합하며, 전반적인 내용을 명확히 다룸
    - 중: 질문과 관련은 있지만 핵심 포인트가 부분적으로 누락됨
    - 하: 질문과 관련이 약하거나 엉뚱한 내용 중심

    2. 답변의 구체성
    - 상: 질문의 핵심적인 내용을 구체적으로 서술함
    - 중: 답변에 부분적으로 두루뭉술한 내용이 섞여있음
    - 하: 답변이 너무 짧거나 답변자가 구체적으로 어떤 행동을 했고 어떤 생각을 했는지 드러나있지 않음

    출력 형식(JSON):
    {{
        "질문과의 관련성": "상/중/하 중 하나",
        "답변의 구체성": "상/중/하 중 하나"
    }}
    '''

    evaluate_prompt = ChatPromptTemplate.from_messages([
        ("system", s_msg),
        ("human", h_msg)
    ])

    evaluate_messages = evaluate_prompt.format_messages(current_question = state["current_question"], current_answer = state["current_answer"])
    evaluate_response = llm.invoke(evaluate_messages)
    evaluate_output = evaluate_response.content.strip()

    try:
        eval_dict = json.loads(evaluate_output)
    except json.JSONDecodeError:
        # 형식이 안 맞으면 fallback 처리
        eval_dict = {"질문과의 관련성": "중", "답변의 구체성": "중"}

    conversation = state.get("conversation", [])
    evaluation = state.get("evaluation", [])

    conversation.append({
        "질문": state["current_question"],
        "답변": state["current_answer"]
    })
    evaluation.append(eval_dict)

    return {
        **state,
        "conversation": conversation,
        "evaluation": evaluation
    }

# 반추 노드: 최근 평가가 적절한지 되돌아보고 판단 (보완 필요 여부 포함)
def reflect_node(state: InterviewState) -> InterviewState:
    llm = ChatOpenAI(model_name='gpt-4.1-mini')

    s_msg = "너는 10년 이상 경력의 면접관이야."
    h_msg = '''
    {current_question} 질문과 {current_answer} 답변 내용을 바탕으로 {evaluation} 이런 평가를 내렸었는데 이 평가가 적절한지 다시 판단해줘.
    평가 기준은 아래와 같았어. 이 기준에 따라 평가가 제대로 됐는지 확인해서 제대로 판단 됐으면 "정상"이라고 하고 다시 평가해야 한다고 판단되면 "재평가 필요"라고 말해줘.
    반드시 "정상" 또는 "재평가 필요" 중 하나만 출력하도록 해.

    평가 기준:
    아래 질문과 답변을 보고 면접 답변이 적절한지 아래 2가지 항목을 각각 상, 중, 하로 평가해줘.

    질문: {current_question}
    답변: {current_answer}

    평가 항목:
    1. 질문과의 관련성
    - 상: 질문의 핵심 의도에 정확히 부합하며, 전반적인 내용을 명확히 다룸
    - 중: 질문과 관련은 있지만 핵심 포인트가 부분적으로 누락됨
    - 하: 질문과 관련이 약하거나 엉뚱한 내용 중심

    2. 답변의 구체성
    - 상: 질문의 핵심적인 내용을 구체적으로 서술함
    - 중: 답변에 부분적으로 두루뭉술한 내용이 섞여있음
    - 하: 답변이 너무 짧거나 답변자가 구체적으로 어떤 행동을 했고 어떤 생각을 했는지 드러나있지 않음
    '''

    # 이력서 핵심 요약
    reflect_prompt = ChatPromptTemplate.from_messages([
        ("system", s_msg),
        ("human", h_msg)
    ])

    reflect_messages = reflect_prompt.format_messages(current_question = state['current_question'], current_answer = state['current_answer'], evaluation = state['evaluation'][-1])
    reflect_response = llm.invoke(reflect_messages)
    reflect_summary = reflect_response.content.strip()

    return {
        **state,
        "reflection_status": reflect_summary
    }

def re_evaluate_answer(state: InterviewState) -> InterviewState:
    # 답변 평가
    llm = ChatOpenAI(model_name='gpt-4.1-mini')

    s_msg = '너는 10년 이상 경력의 면접관이야.'
    h_msg = '''
    아래 질문과 답변을 보고 면접 답변이 적절한지 아래 2가지 항목을 각각 상, 중, 하로 다시 평가해줘. 평가 항목과 기준에 맞게 정확하게 제대로 평가해줘.

    질문: {current_question}
    답변: {current_answer}

    평가 항목:
    1. 질문과의 관련성
    - 상: 질문의 핵심 의도에 정확히 부합하며, 전반적인 내용을 명확히 다룸
    - 중: 질문과 관련은 있지만 핵심 포인트가 부분적으로 누락됨
    - 하: 질문과 관련이 약하거나 엉뚱한 내용 중심

    2. 답변의 구체성
    - 상: 질문의 핵심적인 내용을 구체적으로 서술함
    - 중: 답변에 부분적으로 두루뭉술한 내용이 섞여있음
    - 하: 답변이 너무 짧거나 답변자가 구체적으로 어떤 행동을 했고 어떤 생각을 했는지 드러나있지 않음

    출력 형식(JSON):
    {{
        "질문과의 관련성": "상/중/하 중 하나",
        "답변의 구체성": "상/중/하 중 하나"
    }}
    '''

    evaluate_prompt = ChatPromptTemplate.from_messages([
        ("system", s_msg),
        ("human", h_msg)
    ])

    evaluate_messages = evaluate_prompt.format_messages(current_question = state["current_question"], current_answer = state["current_answer"])
    evaluate_response = llm.invoke(evaluate_messages)
    evaluate_output = evaluate_response.content.strip()

    try:
        eval_dict = json.loads(evaluate_output)
    except json.JSONDecodeError:
        # 형식이 안 맞으면 fallback 처리
        eval_dict = {"질문과의 관련성": "중", "답변의 구체성": "중"}

    evaluation = state.get("evaluation", [])
    if evaluation:
        evaluation[-1] = eval_dict  # 기존 마지막 평가 덮어쓰기
    else:
        evaluation.append(eval_dict)  # 리스트가 비어있으면 새로 추가

    return {
        **state,
        "evaluation": evaluation
    }

# 3) 인터뷰 진행 검토 --------------------
def decide_next_step(state: InterviewState) -> InterviewState:
    conv = state.get('conversation', [])
    if len(conv) >= 5:
        return {**state, 'next_step': 'end'}
    evals = state.get('evaluation', [])
    recent = evals[-1] if evals else None
    if isinstance(recent, dict):
        grades = list(recent.values())
    elif isinstance(recent, (list, tuple, set)):
        grades = list(recent)
    elif recent is None:
        grades = []
    else:
        grades = [recent]
    strategies=['경력 및 경험','동기 및 커뮤니케이션','논리적 사고']
    selected_strategy=state['current_strategy']
    remain=[x for x in strategies if x not in selected_strategy]
    choice=random.choice(remain)
    if any(g == '상' for g in grades):
        return {**state, 'next_step': '',
        'current_strategy':choice
        }
    if any(g == '하' for g in grades):
        return {**state, 'next_step': '',
        'current_strategy':choice
        }
    if grades and all(g == '중' for g in grades):
        return {**state, 'next_step': ''}
    return {**state, 'next_step': ''}

# 4) 질문 생성 --------------------
def generate_question(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.
    llm = ChatOpenAI(model_name='gpt-4.1-mini')
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

    # --------------------------------크로마db-------------------------------------#
    DB_DIRECTORY = "/content/drive/MyDrive/kt aivle/한경훈 대위햄/chroma_db"

    if os.path.exists(DB_DIRECTORY): # chroma db가 존재할 경우
      vectorstore = Chroma(
          persist_directory=DB_DIRECTORY,
          embedding_function=embedding_model
      )

    else: # chroma db가 없을 경우, 직접 생성
      s_prompt_1 = ChatPromptTemplate.from_messages([
          ("system", """너는 이력서 분석 전문가야.
          반드시 아래 요청된 JSON 형식을 따르세요:

          {{
              "경력 및 경험": {{
                  "유사 질문1": "...",
                  "유사 질문2": "...",
                  "유사 질문3": "..."
              }},
              "동기 및 커뮤니케이션": {{
                  "유사 질문1": "...",
                  "유사 질문2": "...",
                  "유사 질문3": "..."
              }},
              "논리적 사고": {{
                  "유사 질문1": "...",
                  "유사 질문2": "...",
                  "유사 질문3": "..."
              }}
          }}
          """),
          ("human", '''{state} 내용 중 이력서 요약, 키워드, 질문전략, 이전 질문과 답변, 평가 등을 기반으로
          지원자의 사고력, 문제 해결 방식, 혹은 기술적 깊이를 더 확인할 수 있도록 추가적인 심화 인터뷰 질문들을 위한 예시 유사 질문들을 3가지 주제에 3개씩 json 형태로 만들어줘. 이전 질문과 중복된 질문은 안돼.
          ''')
      ])

      s_messages_1 = s_prompt_1.format_messages(state=state)
      response_1 = llm.invoke(s_messages_1)
      resume_evaluation = response_1.content.strip()

      try:
          data = json.loads(resume_evaluation)
          question_list: List[str] = []
          for category, questions in data.items():
              for key, question in questions.items():
                  question_list.append(question)
      except json.JSONDecodeError:
              question_list = [resume_evaluation]
      vectorstore = Chroma.from_texts(question_list, embedding_model)#, persist_directory="./chroma_db")

    # --------------------------------db로부터 검색-------------------------------------#

    query = state.get("current_strategy", "")
    keyword = state.get("resume_keywords", [])

    query += ", ".join(keyword)

    retrieved_docs = vectorstore.similarity_search(query, k=3)

    content = vectorstore.get(limit=100, include=["documents"])

    reference_questions = "\n".join([f"- {doc.page_content}" for doc in retrieved_docs])

    # --------------------------------최종 질문 생성-------------------------------------#

    s_prompt = ChatPromptTemplate.from_messages([
        ("system", "넌 이력서 분석 전문가야."),
        ("human", '''{state} 내용 중 이력서 요약, 키워드, 질문전략, 이전 질문과 답변, 평가, 그리고 {similar_questions}의 유사 질문 등을 기반으로
    지원자의 사고력, 문제 해결 방식, 혹은 기술적 깊이를 더 확인할 수 있도록 추가적인 심화 인터뷰 질문을 하나 만들어줘. 질문 외의 다른 설명이나 문장은 필요 없어.
    유사 질문은 참고만 해야하고, 유사질문과 같은 질문이 나오면 안돼.질문은 {strategy} 관련한 질문이여야해. 이전 질문과 중복된 질문은 안돼.
    ''')
    ])
    s_messages = s_prompt.format_messages(state=state, similar_questions=reference_questions, strategy=state["current_strategy"])
    response = llm.invoke(s_messages)

    # return 코드는 제공합니다.

    return {
        **state,
        "current_question": response.content.strip(),
        "current_answer": ""
    }


# 5) 인터뷰 피드백 보고서 --------------------
def summarize_interview(state: InterviewState) -> InterviewState:

    print('\n' + '='*80)
    print('[인터뷰 종료 리포트 - 상/중/하 기본 평가 + STAR 상세 평가]')
    print('='*80)
    # 1. 질문별 답변 및 평가 출력
    for i, conv in enumerate(state['conversation']):
        print(f"\n[질문 {i+1}] {conv['질문']}")
        print(f"[답변 {i+1}] {conv['답변']}")

        eval_data = state["evaluation"][i]

        # 기본 평가 (상/중/하)
        basic_eval = eval_data.get("기본평가", {})
        print(f"\n📊 [기본 평가]")
        print(f"  - 질문 관련성: {basic_eval.get('질문과의_관련성', 'N/A')}")
        print(f"  - 답변 구체성: {basic_eval.get('답변의_구체성', 'N/A')}")
        if '기본평가_근거' in basic_eval:
            print(f"  - 근거: {basic_eval['기본평가_근거']}")

        # STAR 상세 평가
        star_eval = eval_data.get("STAR평가", {})
        print(f"\n⭐ [STAR 상세 평가]")
        print(f"  - S(상황): {star_eval.get('S_상황설명', 'N/A')}점")
        print(f"  - T(역할): {star_eval.get('T_역할목표', 'N/A')}점")
        print(f"  - A(행동): {star_eval.get('A_행동결정', 'N/A')}점")
        print(f"  - R(결과): {star_eval.get('R_결과성과', 'N/A')}점")
        print(f"  - 협동성: {star_eval.get('협동성', 'N/A')}점")
        if 'STAR평가_근거' in star_eval:
            print(f"  - 근거: {star_eval['STAR평가_근거']}")

        print('-'*80)


    # 2. 전체 평균 - STAR 점수 계산
    print('\n' + '='*80)
    print('[STAR 종합 평가 - 전체 평균]')
    print('='*80)

    categories = ['S_상황설명', 'T_역할목표', 'A_행동결정', 'R_결과성과', '협동성']
    category_names = ['S (상황)', 'T (역할)', 'A (행동)', 'R (결과)', '협동성']

    avg_scores = {}
    for cat in categories:
        scores = [eval_data.get("STAR평가", {}).get(cat, 3) for eval_data in state["evaluation"]]
        avg_scores[cat] = round(sum(scores) / len(scores), 1)

    for cat, display_name in zip(categories, category_names):
        print(f"{display_name}: {avg_scores[cat]}점")

    # 3. 상/중/하 빈도 분석
    print('\n' + '='*80)
    print('[기본 평가 요약]')
    print('='*80)

    relevance_counts = {"상": 0, "중": 0, "하": 0}
    specificity_counts = {"상": 0, "중": 0, "하": 0}

    for eval_data in state["evaluation"]:
        basic = eval_data.get("기본평가", {})
        rel = basic.get("질문과의_관련성", "중")
        spec = basic.get("답변의_구체성", "중")
        relevance_counts[rel] = relevance_counts.get(rel, 0) + 1
        specificity_counts[spec] = specificity_counts.get(spec, 0) + 1

    print(f"질문 관련성: 상 {relevance_counts['상']}개, 중 {relevance_counts['중']}개, 하 {relevance_counts['하']}개")
    print(f"답변 구체성: 상 {specificity_counts['상']}개, 중 {specificity_counts['중']}개, 하 {specificity_counts['하']}개")

    # 4. 레이더 차트 생성 (STAR 기준)
    try:
        values = [avg_scores[cat] for cat in categories]

        # 레이더 차트 설정
        N = len(categories)
        angles = [n / float(N) * 2 * pi for n in range(N)]
        values += values[:1]
        angles += angles[:1]

        fig, ax = plt.subplots(figsize=(10, 8), subplot_kw=dict(projection='polar'))

        # 차트 그리기
        ax.plot(angles, values, 'o-', linewidth=2.5, color='#2E86DE', label='STAR 평가')
        ax.fill(angles, values, alpha=0.3, color='#2E86DE')

        # 카테고리 레이블
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(category_names, size=12, weight='bold')

        # Y축 설정
        ax.set_ylim(0, 5)
        ax.set_yticks([1, 2, 3, 4, 5])
        ax.set_yticklabels(['1', '2', '3', '4', '5'], size=10)
        ax.grid(True, linestyle='--', alpha=0.7, linewidth=1.2)

        # 제목
        plt.title('STAR + 협동성 상세 평가\n(5점 만점)',
                 size=16, weight='bold', pad=30)

        # 한글 폰트 설정
        plt.rcParams['font.family'] = 'DejaVu Sans'
        plt.rcParams['axes.unicode_minus'] = False

        # 범례
        ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)

        plt.tight_layout()
        plt.show()

        print("\n✅ STAR 레이더 차트가 생성되었습니다.")

    except Exception as e:
        print(f"❌ 레이더 차트 생성 실패: {str(e)}")
        import traceback
        traceback.print_exc()

    # 5. 최종 종합 피드백
    print('\n' + '='*80)
    print('[최종 종합 피드백]')
    print('='*80)

    # 전체 면접 내용 텍스트화
    conversation_text = "\n\n".join([
        f"질문 {i+1}: {conv['질문']}\n답변 {i+1}: {conv['답변']}"
        for i, conv in enumerate(state['conversation'])
    ])

    # 평가 결과 텍스트화
    basic_summary = f"질문 관련성(상 {relevance_counts['상']}, 중 {relevance_counts['중']}, 하 {relevance_counts['하']}), 답변 구체성(상 {specificity_counts['상']}, 중 {specificity_counts['중']}, 하 {specificity_counts['하']})"
    star_summary = "\n".join([
        f"- {display_name}: {avg_scores[cat]}점"
        for cat, display_name in zip(categories, category_names)
    ])

    # LLM 종합 피드백
    llm = ChatOpenAI(model_name='gpt-4.1-mini')

    feedback_prompt = f'''
아래 면접 내용과 평가 결과를 바탕으로 지원자에게 최종 피드백을 작성해줘.

=== 면접 질문 및 답변 ===
{conversation_text}

=== 기본 평가 (상/중/하) ===
{basic_summary}

=== STAR 상세 평가 (평균) ===
{star_summary}

다음 내용을 포함해서 작성해줘:

1. 기본 평가 종합
   - 질문 관련성과 답변 구체성에 대한 전반적 평가
   - 강점과 개선점

2. STAR 각 항목별 분석
   - S (상황): 상황 설명의 강점과 개선점
   - T (역할): 역할/목표 제시의 강점과 개선점
   - A (행동): 행동/의사결정의 강점과 개선점
   - R (결과): 결과/성과 제시의 강점과 개선점
   - 협동성: 팀워크 및 협업 능력의 강점과 개선점

3. 전체 총평
   - 지원자의 핵심 강점 (2-3가지)
   - 우선 개선이 필요한 영역 (2-3가지)
   - STAR 기법 활용 관점에서의 조언
   - 면접관으로서의 종합 의견
   - 격려의 한마디

전문적이면서도 따뜻한 톤으로 작성하고,
지원자가 실제로 활용할 수 있는 구체적인 조언을 포함해줘.
'''

    feedback_messages = [
        SystemMessage(content="너는 경력 10년 이상의 STAR 면접 피드백 전문가야."),
        HumanMessage(content=feedback_prompt)
    ]

    feedback_response = llm.invoke(feedback_messages)
    final_report = feedback_response.content.strip()

    print('\n' + feedback_response.content.strip())

    print('\n' + '='*80)
    print('인터뷰 피드백 보고서 완료')
    print('='*80 + '\n')

    #pdf 다운로드 !pip reportlab 설치필요
    pdfmetrics.registerFont(UnicodeCIDFont('HYSMyeongJo-Medium'))
    pdf_path = "/content/interview_report.pdf"
    doc = SimpleDocTemplate(pdf_path, pagesize=A4)
    styles = getSampleStyleSheet()
    styles = getSampleStyleSheet()
    styles["Normal"].fontName = "HYSMyeongJo-Medium"
    styles["Title"].fontName = "HYSMyeongJo-Medium"
    elements = []
    elements.append(Paragraph("AI 인터뷰 평가 보고서", styles["Title"]))
    elements.append(Spacer(1, 20))
    elements.append(Paragraph(final_report.replace("\n", "<br/>"), styles["Normal"]))
    elements.append(Spacer(1, 20))
    chart_path = "/content/star_chart.png"
    plt.savefig(chart_path)
    elements.append(Image(chart_path, width=400, height=350))
    doc.build(elements)
    plt.close()
    return {
        **state,
        "final_report": final_report,   # ✅ Gradio가 접근 가능하도록 저장
        "pdf_path" : pdf_path
    }


# 6) Agent --------------------
# 분기 판단 함수
def route_next(state: InterviewState) -> Literal["generate", "summarize"]:
    print(state['next_step'])
    return "" if state["next_step"] == "" else "end"

def check_satisfaction(state: InterviewState):
    return "재평가 필요" if state["reflection_status"] == "재평가 필요" else "정상"

# 그래프 정의 시작
builder = StateGraph(InterviewState)

# 노드 추가
builder.add_node("evaluate", evaluate_answer)
builder.add_node("reflect", reflect_node)
builder.add_node("re_evaluate", re_evaluate_answer)
builder.add_node("decide", decide_next_step)
builder.add_node("generate", generate_question)
builder.add_node("summarize", summarize_interview)

# 노드 연결
builder.add_conditional_edges("reflect", check_satisfaction,
                              {"정상": "decide", "재평가 필요": "re_evaluate"})
builder.add_conditional_edges("decide", route_next,
                              {"": "generate", "end": "summarize"}
                              )

builder.set_entry_point("evaluate")
builder.add_edge("evaluate", "reflect")
builder.add_edge("re_evaluate", "decide")
builder.set_finish_point("generate")
builder.set_finish_point("summarize")

# 컴파일
graph = builder.compile()
#-------------------------------------------------------------------


########### 다음 코드는 제공되는 gradio 코드 입니다.################

import gradio as gr
import tempfile

# 세션 상태 초기화 함수
def initialize_state():
    return {
        "state": None,
        "interview_started": False,
        "interview_ended": False,
        "chat_history": []
    }

# 파일 업로드 후 인터뷰 초기화
def upload_and_initialize(file_obj, session_state):
    if file_obj is None:
        return session_state, "파일을 업로드해주세요."

    # Gradio는 file_obj.name 이 파일 경로야
    file_path = file_obj.name

    # 인터뷰 사전 처리
    state = preProcessing_Interview(file_path)
    session_state["state"] = state
    session_state["interview_started"] = True

    # 첫 질문 저장
    first_question = state["current_question"]
    session_state["chat_history"].append(["🤖 AI 면접관", first_question])

    return session_state, session_state["chat_history"]

# 답변 처리 및 다음 질문 생성
def chat_interview(user_input, session_state):
    if not session_state["interview_started"]:
        return session_state, "먼저 이력서를 업로드하고 인터뷰를 시작하세요.", gr.update(value=""), None

    # (1) 사용자 답변 저장
    session_state["chat_history"].append(["🙋‍♂️ 지원자", user_input])
    session_state["state"] = update_current_answer(session_state["state"], user_input)

    # (2) Agent 실행 (평가 및 다음 질문 or 종료)
    session_state["state"] = graph.invoke(session_state["state"])

    # (3) 종료 여부 판단
    if session_state["state"]["next_step"] == "end":
        session_state["interview_ended"] = True
        final_summary = "✅ 인터뷰가 종료되었습니다!\n\n"
        final_report = session_state["state"].get("final_report", "⚠️ 보고서 생성 오류")
        pdf_path = session_state["state"].get("pdf_path", None)
        for i, turn in enumerate(session_state["state"]["conversation"]):
            final_summary += f"\n**[질문 {i+1}]** {turn['질문']}\n**[답변 {i+1}]** {turn['답변']}\n"
            if i < len(session_state["state"]["evaluation"]):
                eval_result = session_state["state"]["evaluation"][i]
                final_summary += f"_평가 - 질문 관련성: {eval_result['질문과의 관련성']}, 답변 구체성: {eval_result['답변의 구체성']}_\n"
        session_state["chat_history"].append(["🤖 AI 면접관", final_report])
        # 대화 끝나면 pdf 다운버튼 활성화
        return session_state, session_state["chat_history"], gr.update(value=""), gr.update(value=pdf_path, visible=True)

    else:
        next_question = session_state["state"]["current_question"]
        session_state["chat_history"].append(["🤖 AI 면접관", next_question])
        return session_state, session_state["chat_history"], gr.update(value=""), None

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    session_state = gr.State(initialize_state())

    gr.Markdown("# 🤖 AI 면접관 \n이력서를 업로드하고 인터뷰를 시작하세요!")

    with gr.Row():
        file_input = gr.File(label="이력서 업로드 (PDF 또는 DOCX)")
        upload_btn = gr.Button("인터뷰 시작")

    chatbot = gr.Chatbot()
    user_input = gr.Textbox(show_label=False, placeholder="답변을 입력하고 Enter를 누르세요.")
    #pdf 다운 버튼
    download_btn = gr.File(label="📄 인터뷰 결과 보고서 다운로드", visible=False)

    #pdf
    upload_btn.click(upload_and_initialize, inputs=[file_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(chat_interview, inputs=[user_input, session_state], outputs=[session_state, chatbot,user_input, download_btn])
    user_input.submit(lambda: "", None, user_input)

# 실행
demo.launch(share=True)

## **3. 실행**

In [ ]:
!python app.py